In [1]:
import pandas as pd
import numpy as np
from numpy import arange,cos,sin,pi,sqrt,arcsin

%matplotlib inline
import matplotlib.pyplot as plt
from astropy import constants as C


In [2]:
#carga de datos planetarios obtenidos en la base de datos de Gaia
exoplanet=pd.read_csv("PSCompPars_2022.04.06_11.27.07.csv")
exoplanet
# COLUMN pl_name:        Planet Name
# COLUMN hostname:       Host Name
# COLUMN discoverymethod: Discovery Method
# COLUMN pl_controv_flag: Controversial Flag
# COLUMN pl_orbper:      Orbital Period [days]
# COLUMN pl_orbsmax:     Orbit Semi-Major Axis [au])
# COLUMN pl_radj:        Planet Radius [Jupiter Radius]
# COLUMN pl_bmassj:      Planet Mass or Mass*sin(i) [Jupiter Mass]
# COLUMN pl_orbeccen:    Eccentricity
# COLUMN pl_orbincl:     Inclination [deg]
# COLUMN pl_tranmid:     Transit Midpoint [days]
# COLUMN ttv_flag:       Data show Transit Timing Variations
# COLUMN pl_rvamp:       Radial Velocity Amplitude [m/s]
# COLUMN st_spectype:    Spectral Type
# COLUMN st_rad:         Stellar Radius [Solar Radius]
# COLUMN st_mass:        Stellar Mass [Solar mass]
# COLUMN rastr:          RA [sexagesimal]
# COLUMN ra:             RA [deg]
# COLUMN decstr:         Dec [sexagesimal]
# COLUMN dec:            Dec [deg]
# COLUMN sy_dist:        Distance [pc]
# COLUMN sy_plx:         Parallax [mas]
# COLUMN sy_gaiamag:     Gaia Magnitude


,pl_name,hostname,discoverymethod,pl_controv_flag,pl_orbper,pl_orbsmax,pl_radj,pl_bmassj,pl_orbeccen,pl_orbincl,...,st_spectype,st_rad,st_mass,rastr,ra,decstr,dec,sy_dist,sy_plx,sy_gaiamag
0,11 Com b,11 Com,Radial Velocity,0,326.030000,1.290000,1.08,19.4000,0.2310,NaN,...,G8 III,19.00,2.70,12h20m42.91s,185.178779,+17d47m35.71s,17.793252,93.1846,10.71040,4.44038
1,11 UMi b,11 UMi,Radial Velocity,0,516.219970,1.530000,1.09,14.7400,0.0800,NaN,...,K4 III,29.79,2.78,15h17m05.90s,229.274595,+71d49m26.19s,71.823943,125.3210,7.95388,4.56216
2,14 And b,14 And,Radial Velocity,0,185.840000,0.830000,1.15,4.8000,0.0000,NaN,...,K0 III,11.00,2.20,23h31m17.80s,352.824150,+39d14m09.01s,39.235837,75.4392,13.22890,4.91781
3,14 Her b,14 Her,Radial Velocity,0,1773.400020,2.930000,1.15,4.6600,0.3700,NaN,...,K0 V,0.93,0.90,16h10m24.50s,242.602101,+43d48m58.90s,43.816362,17.9323,55.73630,6.38300
4,16 Cyg B b,16 Cyg B,Radial Velocity,0,798.500000,1.660000,1.20,1.7800,0.6800,NaN,...,G3 V,1.13,1.08,19h41m51.75s,295.465642,+50d31m00.57s,50.516824,21.1397,47.27540,6.06428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,ups And b,ups And,Radial Velocity,0,4.617033,0.059222,1.25,0.6876,0.0215,NaN,...,F8 V,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687
913,ups And c,ups And,Radial Velocity,0,241.258000,0.827774,1.10,13.9800,0.2596,NaN,...,F8 V,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687
914,ups And d,ups And,Radial Velocity,0,1276.460000,2.513290,1.11,10.2500,0.2987,NaN,...,F8 V,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687
915,ups Leo b,ups Leo,Radial Velocity,0,385.200000,1.180000,1.27,0.5100,0.3200,NaN,...,G9 III,11.22,1.48,11h36m56.93s,174.237219,-00d49m24.83s,-0.823564,52.5973,18.99390,4.03040


In [3]:
#describimos las constantes que se van a ncesitar
Pseg=exoplanet.pl_orbper*24*3600 #periodo orbital en dias
mtot=exoplanet.st_mass*C.M_sun.value #masa de la estrella en masas solares
dtot=exoplanet.sy_dist*C.pc.value #distancia al sol en metros
m2sini=exoplanet.pl_bmassj #masa mínima del planeta en masas de jupiter
e=exoplanet.pl_orbeccen #excenricidad de la órbita

#Constantes
G=C.G.value

e=0 #suponiendo orbita circular
omega2=pi/2
u=len(exoplanet.index)  #numero de planetas detectados por velocidad radial
U=arange(u)


In [4]:
G

6.6743e-11

In [5]:
#lo primero de todo es definir un limite inferior para i para cada planeta.
i_min=[]
for l in U:
    i_min.append(arcsin(m2sini[l]/72
                       ))


In [6]:
#DEFINIMOS EL ARRAY DE LA INCLINACION CON EL VALOR MINIMO OBTENIDO EN EL CÓDIGO ANTERIOR. CADA PLANETA TIENE SU PROPIO ARRAY
#tiene que salir una lista de len=len(u) y en cada posición un array de i con i_min comenzando
i=[]
for o in U:
    i.append(np.arange(i_min[o],pi/2,0.001))


In [7]:
m2_jup=[]
for j in U:
    m2_jup.append(m2sini[j]/sin(i[j]))


In [8]:
# from matplotlib.backends.backend_pdf import PdfPages

# plt.rcParams.update({'figure.max_open_warning': 0})
# fig=[]
# for m in U:
#     plt.figure()
#     img=plt.plot(i[m]*180/pi,m2_jup[m],label='Curva relación Masa-Inclinación')
#     plt.ylim(m2sini[m],80)
#     plt.title(exoplanet.pl_name[m])
#     plt.ylabel('Masa planetaria ($M_J$)')
#     plt.xlabel('Inclinación(deg)')
#     plt.axhline(y=13,color='red',label='Masa límite enana marrón')
#     plt.axhline(y=72,color='green',label='Masa límite estrella')
#     plt.legend()
#     fig.append(img)
    
    
   


In [9]:
#para los siguientes calculos pasamos la masa del planeta a kg
m2_kg=[]
for k in U:
    m2_kg.append(m2_jup[k]*C.M_jup.value)

In [10]:
#AHORA PROCEDEMOS A DEFINIR RHO 
def rho1(a,m2,mt,inc):
    return a*m2/mt/sin(inc)

In [11]:
#caclulo de el semieje mayor total: la suma de los semiejes del planeta y la estrella. Se usa la tercera ley de kepler

def atotal(P,mtot):
    return (P**2*G*mtot/(4*pi**2))**(1/3)

In [12]:
atot=atotal(Pseg,mtot)/dtot*180/pi*3600e6   #sale en muas


In [13]:
#CONVERSION PARA LA ESTRELLA UTILIZANDO EL FACTOR M1/M2
ATOT=[]
for n in U:
    ATOT.append(atot[n]*m2_kg[n]/mtot[n])
ATOT

[array([352.64473685, 351.38902037, 350.1425639 , ...,  95.01858717,
         95.01835151,  95.01821086]),
 array([349.34551373, 347.68334048, 346.03725386, ...,  71.51903469,
         71.51888333,  71.51880348]),
 array([343.26771174, 338.20607663, 333.29187298, ...,  22.88456363,
         22.88452747,  22.88451419]),
 array([11789.42133994, 11610.41491033, 11436.7743849 , ...,
          763.03911823,   763.03795167,   763.03754815]),
 array([5202.60645476, 5000.40714875, 4813.34144129, ...,  128.62026889,
         128.62006676,  128.61999324]),
 array([654.49744449, 643.78731069, 633.42267582, ...,  39.26999627,
         39.26990751,  39.26985802]),
 array([1008.26280151, 1001.33566891,  994.50405839, ...,  144.23795976,
         144.23770759,  144.23759966]),
 array([137.26647833, 127.20293913, 118.51431261, ...,   1.7348998 ,
          1.73489693,   1.73489579]),
 array([825.26555371, 796.46031101, 769.59884059, ...,  22.80947585,
         22.80943813,  22.80942321]),
 array([1288.

In [14]:
#CÁLCULO DE RHO. Tiene que salir para cada planeta un array con los valores de rho para cada inclinación. 
rho=[]
for p in U:
    rho.append(rho1(atot[p],m2sini[p]*C.M_jup.value,mtot[p],i[p]))


$\rho$ corresponde al semieje mayor de la orbita de la estrella entorno al centro de masas. En este código se obtiene un limite superior para él.

In [15]:
#sacamos el rho maximo para cada planeta y los guardamos en un data frame
max_rho=[]
for v in U:
    max_rho.append(np.amax(rho[v]))


In [16]:
Rhomax=pd.DataFrame(max_rho)

# Rhomax.set_axis(['Rhomax'],axis=1)
# Rhomax.rename(index=exoplanet.pl_name)
Rhomax.columns=['Rhomax']
Rhomax.index=exoplanet.pl_name
Rhomax

,Rhomax
pl_name,
11 Com b,352.644737
11 UMi b,349.345514
14 And b,343.267712
14 Her b,11789.421340
16 Cyg B b,5202.606455
...,...
ups And b,233.571039
ups And c,3264.611368
ups And d,9912.468337


In [17]:
new_exoplanet = pd.merge(exoplanet, Rhomax,on='pl_name')
new_exoplanet

,pl_name,hostname,discoverymethod,pl_controv_flag,pl_orbper,pl_orbsmax,pl_radj,pl_bmassj,pl_orbeccen,pl_orbincl,...,st_rad,st_mass,rastr,ra,decstr,dec,sy_dist,sy_plx,sy_gaiamag,Rhomax
0,11 Com b,11 Com,Radial Velocity,0,326.030000,1.290000,1.08,19.4000,0.2310,NaN,...,19.00,2.70,12h20m42.91s,185.178779,+17d47m35.71s,17.793252,93.1846,10.71040,4.44038,352.644737
1,11 UMi b,11 UMi,Radial Velocity,0,516.219970,1.530000,1.09,14.7400,0.0800,NaN,...,29.79,2.78,15h17m05.90s,229.274595,+71d49m26.19s,71.823943,125.3210,7.95388,4.56216,349.345514
2,14 And b,14 And,Radial Velocity,0,185.840000,0.830000,1.15,4.8000,0.0000,NaN,...,11.00,2.20,23h31m17.80s,352.824150,+39d14m09.01s,39.235837,75.4392,13.22890,4.91781,343.267712
3,14 Her b,14 Her,Radial Velocity,0,1773.400020,2.930000,1.15,4.6600,0.3700,NaN,...,0.93,0.90,16h10m24.50s,242.602101,+43d48m58.90s,43.816362,17.9323,55.73630,6.38300,11789.421340
4,16 Cyg B b,16 Cyg B,Radial Velocity,0,798.500000,1.660000,1.20,1.7800,0.6800,NaN,...,1.13,1.08,19h41m51.75s,295.465642,+50d31m00.57s,50.516824,21.1397,47.27540,6.06428,5202.606455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,ups And b,ups And,Radial Velocity,0,4.617033,0.059222,1.25,0.6876,0.0215,NaN,...,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687,233.571039
913,ups And c,ups And,Radial Velocity,0,241.258000,0.827774,1.10,13.9800,0.2596,NaN,...,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687,3264.611368
914,ups And d,ups And,Radial Velocity,0,1276.460000,2.513290,1.11,10.2500,0.2987,NaN,...,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687,9912.468337
915,ups Leo b,ups Leo,Radial Velocity,0,385.200000,1.180000,1.27,0.5100,0.3200,NaN,...,11.22,1.48,11h36m56.93s,174.237219,-00d49m24.83s,-0.823564,52.5973,18.99390,4.03040,1042.490109


In [18]:
#vamos ha coger unicamente los planetas que si puedan ser detectados por Gaia con un maxRho > 30muas
gaialim=20

        
indexNames = new_exoplanet[ new_exoplanet['Rhomax']<gaialim ].index
# Delete these row indexes from dataFrame
new_exoplanet.drop(indexNames , inplace=True)
new_exoplanet   

,pl_name,hostname,discoverymethod,pl_controv_flag,pl_orbper,pl_orbsmax,pl_radj,pl_bmassj,pl_orbeccen,pl_orbincl,...,st_rad,st_mass,rastr,ra,decstr,dec,sy_dist,sy_plx,sy_gaiamag,Rhomax
0,11 Com b,11 Com,Radial Velocity,0,326.030000,1.290000,1.08,19.4000,0.2310,NaN,...,19.00,2.70,12h20m42.91s,185.178779,+17d47m35.71s,17.793252,93.1846,10.71040,4.44038,352.644737
1,11 UMi b,11 UMi,Radial Velocity,0,516.219970,1.530000,1.09,14.7400,0.0800,NaN,...,29.79,2.78,15h17m05.90s,229.274595,+71d49m26.19s,71.823943,125.3210,7.95388,4.56216,349.345514
2,14 And b,14 And,Radial Velocity,0,185.840000,0.830000,1.15,4.8000,0.0000,NaN,...,11.00,2.20,23h31m17.80s,352.824150,+39d14m09.01s,39.235837,75.4392,13.22890,4.91781,343.267712
3,14 Her b,14 Her,Radial Velocity,0,1773.400020,2.930000,1.15,4.6600,0.3700,NaN,...,0.93,0.90,16h10m24.50s,242.602101,+43d48m58.90s,43.816362,17.9323,55.73630,6.38300,11789.421340
4,16 Cyg B b,16 Cyg B,Radial Velocity,0,798.500000,1.660000,1.20,1.7800,0.6800,NaN,...,1.13,1.08,19h41m51.75s,295.465642,+50d31m00.57s,50.516824,21.1397,47.27540,6.06428,5202.606455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,ups And b,ups And,Radial Velocity,0,4.617033,0.059222,1.25,0.6876,0.0215,NaN,...,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687,233.571039
913,ups And c,ups And,Radial Velocity,0,241.258000,0.827774,1.10,13.9800,0.2596,NaN,...,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687,3264.611368
914,ups And d,ups And,Radial Velocity,0,1276.460000,2.513290,1.11,10.2500,0.2987,NaN,...,1.56,1.30,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,74.57110,3.98687,9912.468337
915,ups Leo b,ups Leo,Radial Velocity,0,385.200000,1.180000,1.27,0.5100,0.3200,NaN,...,11.22,1.48,11h36m56.93s,174.237219,-00d49m24.83s,-0.823564,52.5973,18.99390,4.03040,1042.490109


In [19]:
new_exoplanet.to_csv('Exoplanetas detectables uno.csv')